#### 1. Getting up 

In [ ]:
# Install and setup environment
!pip install langchain
!pip install openai
!pip install langchain-google-genai 
!pip install langchain-community 
!pip install langchain-text-splitters  
!export OPENAI_API_KEY="your-openai-api-key" 
!export GOOGLE_API_KEY="your-google-api-key" 


In [ ]:
import os 
from dotenv import load_dotenv
load_dotenv()

#### 2. Using LLMs in LangChain 

Useful parameters to configure llms 
* __temperature__ : lower value produce more predictable output and higher value generate more creative, unexpected.   
* __max_token__ : Limit size(and cost) of the output. 

In [31]:
### Google Gemini 2.5 Flash with LangChain 
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0)

llm.invoke("Who is Trump ?") 

AIMessage(content='Donald John Trump (born 1946) is an American businessman, television personality, and politician who served as the **45th President of the United States from 2017 to 2021**.\n\nHere\'s a breakdown of his background and career:\n\n1.  **Early Life and Business Career:**\n    *   Born in Queens, New York City, he inherited a real estate business from his father, Fred Trump.\n    *   He expanded it into a global empire, developing hotels, casinos, golf courses, and other properties under the Trump Organization.\n    *   He also became a prominent media figure, particularly known for hosting the reality television show "The Apprentice" for over a decade, which significantly boosted his public profile.\n\n2.  **Political Career and Presidency (2017-2021):**\n    *   Trump\'s political career began with his successful campaign for president in 2016, running as a Republican. His campaign was characterized by populist rhetoric, promises to "Make America Great Again," and a f

Some LLM provider (OpenAI) differentiate message to 3 roles: 

* __system role__ : used for instruction the model should to do

* __user role__ : used for the user'query and any other content produce by user

* __assistant role__ : used for content generated by the model

Alternatively, ChatModel make use of different type of chat message interfaces associated with each role: 

* __HumanMessage__ 
* __AIMessasge__ 
* __SystemMessage__ 
* __ChatMessage__ 

In [32]:
from langchain_core.messages import SystemMessage, HumanMessage
systemMsg = SystemMessage(
    '''
    You are a helpful assistant that response to questions with 
    three exclamation marks at the end.
    '''
)
humanMsg = HumanMessage(
    '''
    What is capital of France?
    '''
)

llm.invoke([systemMsg, humanMsg])




AIMessage(content='Paris!!!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--7a3331d0-f9f0-4be7-a3fd-4480b0e14bf6-0', usage_metadata={'input_tokens': 35, 'output_tokens': 34, 'total_tokens': 69, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 32}})

#### 3. Prompt Template

In [36]:
from langchain_core.prompts import PromptTemplate 

template = PromptTemplate.from_template( 
    """
    You are a helpful assistant that response to questions based on the given context.
    If cant answer based on the context, say "I don't know".

    Context: {context} 

    Question: {question}

    Answer:
    """,
)
prompt = template.invoke(
    {
        "context": "The capital of France is Paris.",
        "question": "What is the capital of VietNam?"
    }
)   
completion = llm.invoke(prompt)
print(completion)

content="I don't know" additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'} id='lc_run--09a8f3f3-29cc-48d3-9c65-8ee853fcc2ab-0' usage_metadata={'input_tokens': 66, 'output_tokens': 195, 'total_tokens': 261, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 190}}


#### 4.Specific Formats out of LLMs 

In [30]:
## Define Format Structures 
# Example: JSON Format 
from langchain_core.utils.pydantic import BaseModel, Field 

class AnswerFormat(BaseModel):
    ''' Answer to the user's question along with justification for the answer.'''
    answer: str
    """The answer to the user's question."""

    justification: str
    """The justification for the answer.""" 

structured_llm = llm.with_structured_output(AnswerFormat)
structured_llm.invoke("""What weighs morre, a pound of bricks or a pound of feathers?""")

AnswerFormat(answer='They both weigh the same.', justification='A pound of bricks weighs one pound, and a pound of feathers also weighs one pound. The weight is the same, only the density and volume differ.')

#### 5. Summary 

* __invoke__ transforms a single input into an output 
* __batch__ efficiently transform multiple input into multiple output 
* __stream__ stream output from a single input ass it's produced 

In [38]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0) 

comlpetion = llm.invoke("Tell me a joke about programming.")
print("Invoke: ", comlpetion) 
completions = llm.batch(
    [
        "Tell me a joke about programming.",
        "Tell me a joke about cats.",
        "Tell me a joke about dogs.",
    ]
)       
print("Batch: ", completions) 
completion_stream = llm.stream("Tell me a joke about AI.")
print("Stream: ", end=" ")      


Invoke:  content="There are 10 types of people in the world: those who understand binary, and those who don't." additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'} id='lc_run--76367ec3-af49-458e-9455-7ceabe5c7653-0' usage_metadata={'input_tokens': 8, 'output_tokens': 1221, 'total_tokens': 1229, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 1197}}
Batch:  [AIMessage(content="There are 10 types of people in the world: those who understand binary, and those who don't.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--c7bf3f63-96dc-414f-b34f-b699892d49eb-0', usage